In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install gdown

In [ ]:
!gdown --id 14Wpnt6DPX93rtcH0RsvFZ3eFgtoA1K8K

In [ ]:
!gdown --id 1AMiOTLuAqRgtZ624TfTHwW7fSSS5GbCh

In [ ]:
print(os.listdir("/kaggle/working"))

In [ ]:
path = '/kaggle/working/'
train_df = pd.read_csv(path+'train.csv')
test_df = pd.read_csv(path+'test.csv')

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = text.lower()
    # Remove stopwords
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Preprocess train and test datasets
train_df['cleaned_text'] = train_df['text'].apply(preprocess_text)
test_df['cleaned_text'] = test_df['text'].apply(preprocess_text)

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

batch_size = 16  # Recommended batch size for running on Google Colab

class NewsDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels
        self.model_name = 'bert-base-uncased'
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        encoded_dict = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        item = {
            'input_ids': encoded_dict['input_ids'].squeeze(0),
            'attention_mask': encoded_dict['attention_mask'].squeeze(0),
        }
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.texts)

# Split the train dataset into train and validation datasets
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df['label'])

# Create train, validation, and test datasets
train_dataset = NewsDataset(train_df['cleaned_text'], train_df['label'])
val_dataset = NewsDataset(val_df['cleaned_text'], val_df['label'])
test_dataset = NewsDataset(test_df['cleaned_text'])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch.nn as nn
from transformers import BertModel, BertConfig

class BertForMultiClassification(nn.Module):
    def __init__(self, num_labels):
        super(BertForMultiClassification, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits

num_labels = len(train_df['label'].unique())
model = BertForMultiClassification(num_labels)

In [ ]:
!pip install tqdm

In [ ]:
import torch
import time
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

# Set up GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

# Define training parameters
epochs = 3
lr = 2e-5
weight_decay = 1e-2

# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print("-" * 10)

    model.train()
    train_loss = 0.0
    train_steps = 0
    start_time = time.time()

    with tqdm(train_loader, desc="Training", unit="batch") as pbar:
        for batch in pbar:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            loss = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss.backward()

            train_loss += loss.item()
            train_steps += 1
            pbar.set_postfix({'train_loss': train_loss / train_steps})

            optimizer.step()
            scheduler.step()
        
    elapsed_time = time.time() - start_time
    print(f"Train loss: {train_loss / train_steps:.4f}, Elapsed time: {elapsed_time:.2f}s")

    # Evaluate on the validation dataset
    model.eval()
    val_loss = 0.0
    val_steps = 0

    with tqdm(val_loader, desc="Validation", unit="batch") as pbar:
        for batch in pbar:
            with torch.no_grad():
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                loss = model(input_ids, attention_mask=attention_mask, labels=labels)

                val_loss += loss.item()
                val_steps += 1
                pbar.set_postfix({'valid_loss': val_loss / val_steps})

    print(f"Validation loss: {val_loss / val_steps:.4f}\n")

In [ ]:
# Inference on test dataset
model.eval()
predictions = []

for batch in tqdm(test_loader, desc="Testing", unit="batch"):
    with torch.no_grad():
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        logits = model(input_ids, attention_mask=attention_mask)  # Changed this line
        _, preds = torch.max(logits, dim=1)

        predictions.extend(preds.cpu().numpy())

# Prepare submission
submission = pd.DataFrame({"id": test_df["id"], "label": predictions})
submission.to_csv("submission.csv", index=False)